In [192]:
import pandas as pd
from datetime import datetime
import numpy as np

In [193]:
df1 = pd.read_csv('./data/CLIENT_191102.tsv', sep = '\t')
df_case = df1.drop(columns = ['EE Provider ID', 'Client Unique ID'])  #EE UID is unique for each case
df_clients = pd.DataFrame(df_case.groupby('Client ID').size()).rename(columns = {0: 'frequency'}) #some client came for more than once
#2364 unique clients
df = df_case.drop(columns = ['EE UID', 'Client Age at Entry', 'Client Age at Exit']).drop_duplicates()
df_clients = pd.merge(df_clients, df, on = 'Client ID', how = 'left')

In [194]:
df2 = pd.read_csv('./data/DISABILITY_ENTRY_191102.tsv', sep = '\t')
df3 = pd.read_csv('./data/DISABILITY_EXIT_191102.tsv', sep = '\t')

df_diab_entry = df2.rename(columns = {'Disability Determination (Entry)': 'disability_determination_entry', 'Disability Type (Entry)': 'disability_type'})
df_diab_entry = df_diab_entry[df_diab_entry.disability_determination_entry == 'Yes (HUD)'] #filter clients with disability when entry

df_diab_exit = df3.rename(columns = {'Disability Determination (Exit)': 'disability_determination_exit', 'Disability Type (Exit)': 'disability_type'})
df_diab_exit = df_diab_exit[df_diab_exit.disability_determination_exit == 'Yes (HUD)'] #filter clients with disability when exit

df_disable = pd.merge(df_diab_exit,  df_diab_entry, on = ['Client ID', 'disability_type'], how = 'left') #join two dataframe
df_disable = df_disable[['Client ID', 'disability_determination_exit', 'disability_type', 'disability_determination_entry']]
df_disable[df_disable.disability_determination_exit != df_disable.disability_determination_entry] #find out the difference between entry and exit
#data show no information in the change of disability

,Client ID,disability_determination_exit,disability_type,disability_determination_entry
15015,373323,Yes (HUD),Mental Health Problem (HUD),NaN
15906,206895,Yes (HUD),Developmental (HUD),NaN
16759,450723,Yes (HUD),Mental Health Problem (HUD),NaN
16784,227895,Yes (HUD),Mental Health Problem (HUD),NaN
16785,227895,Yes (HUD),Mental Health Problem (HUD),NaN
17859,441891,Yes (HUD),Both Alcohol and Drug Abuse (HUD),NaN
18216,421698,Yes (HUD),Mental Health Problem (HUD),NaN
18275,398931,Yes (HUD),Both Alcohol and Drug Abuse (HUD),NaN
18640,420015,Yes (HUD),Drug Abuse (HUD),NaN
18832,462367,Yes (HUD),Drug Abuse (HUD),NaN


In [195]:
df_disable_indicator = pd.DataFrame(df_diab_exit.groupby('Client ID').size()).rename(columns = {0: 'disability_size'})
df_clients = pd.merge(df_clients, df_disable_indicator, on = 'Client ID', how = 'left')
df_clients = df_clients.fillna(0)
df_clients['disability'] = round(df_clients['disability_size']/df_clients['frequency'])
df_clients = df_clients.drop(columns = ['disability_size'])

In [196]:
df_entry_exit = pd.read_csv('./data/ENTRY_EXIT_191102.tsv', sep = '\t')
df_entry_exit = df_entry_exit[['EE UID', 'Client ID', 'Entry Date', 'Exit Date', 'Destination', 'Reason for Leaving']]
df_entry_exit['Entry Date'] = pd.to_datetime(df_entry_exit['Entry Date'], format = '%m/%d/%Y')
df_entry_exit['Exit Date'] = pd.to_datetime(df_entry_exit['Exit Date'], format = '%m/%d/%Y')
df_entry_exit['Exit Date'] = df_entry_exit['Exit Date'].replace(np.datetime64('NaT'), datetime.now())
df_entry_exit['Period'] = (df_entry_exit['Exit Date'] - df_entry_exit['Entry Date']).dt.days

In [197]:
df = df_entry_exit.groupby('Client ID').mean().drop(columns = ['EE UID'])
df_clients = pd.merge(df_clients, df, on = 'Client ID', how = 'left')

In [198]:
df_entry_exit['Entry_Year'] = df_entry_exit['Entry Date'].map(lambda x: x.year)
df_entry_exit['Entry_Month'] = df_entry_exit['Entry Date'].map(lambda x: x.month)
df_entry_exit['Day'] = df_entry_exit['Entry Date'].map(lambda x: x.day)
df_entry_exit['Exit_Year'] = df_entry_exit['Exit Date'].map(lambda x: x.year)
df_entry_exit['Exit_Month'] = df_entry_exit['Exit Date'].map(lambda x: x.month)
df_entry_exit['Exit_Day'] = df_entry_exit['Exit Date'].map(lambda x: x.day)

In [199]:
df_nclient = pd.DataFrame(df_entry_exit.groupby('Entry_Year', as_index=False).size()).rename(columns = {0: 'number_cases'})
df_year = df_entry_exit[['Client ID', 'Entry_Year']]
df_ncase = df_year.groupby(['Entry_Year'], as_index=False).agg({"Client ID": pd.Series.nunique}).rename(columns = {'Client ID': 'number_clients'})
df_n = pd.merge(df_nclient, df_ncase, on = ['Entry_Year'], how = 'left')
df_n = df_n[df_n['Entry_Year'] > 2012]

In [200]:
df6 = pd.read_csv('./data/HEALTH_INS_ENTRY_191102.tsv', sep = '\t')
df7 = pd.read_csv('./data/HEALTH_INS_EXIT_191102.tsv', sep = '\t')

In [201]:
l = [] 
for Ins_type, sub_df in df6.groupby('Health Insurance Type (Entry)'):
    l.append(Ins_type)
print(l)

ls = []
for Ins_type, sub_df in df7.groupby('Health Insurance Type (Exit)'):
    ls.append(Ins_type)
print(ls)
l == ls
#type of health insurance are same

['Employer - Provided Health Insurance', 'Health Insurance obtained through COBRA', 'Indian Health Services Program', 'MEDICAID', 'MEDICARE', 'Other', 'Private Pay Health Insurance', "State Children's Health Insurance Program", 'State Health Insurance for Adults', "Veteran's Administration (VA) Medical Services"]
['Employer - Provided Health Insurance', 'Health Insurance obtained through COBRA', 'Indian Health Services Program', 'MEDICAID', 'MEDICARE', 'Other', 'Private Pay Health Insurance', "State Children's Health Insurance Program", 'State Health Insurance for Adults', "Veteran's Administration (VA) Medical Services"]


True

In [202]:
df6s = df6[df6['Covered (Entry)'] == 'Yes'].rename(columns = {'Health Insurance Type (Entry)': 'health_insurance_type'})
df6s = df6s[['Client ID', 'Covered (Entry)', 'health_insurance_type']]
df7s = df7[df7['Covered (Exit)'] == 'Yes'].rename(columns = {'Health Insurance Type (Exit)': 'health_insurance_type'})
df7s = df7s[['Client ID', 'Covered (Exit)', 'health_insurance_type']]
df_healthins = pd.merge(df7s, df6s, on = ['Client ID', 'health_insurance_type'], how = 'inner').drop_duplicates()
df_healthins[df_healthins['Covered (Exit)'] != df_healthins['Covered (Entry)']]
#no difference in health insurance covered when entry and exit

,Client ID,Covered (Exit),health_insurance_type,Covered (Entry)


In [203]:
df_healthins_types = pd.DataFrame(df_healthins.groupby('Client ID').size()).rename(columns = {0: 'healthins_types'})
df_clients = pd.merge(df_clients, df_healthins_types, on = ['Client ID'], how = 'left')  #add health insurance

In [204]:
df8 = pd.read_csv('./data/INCOME_ENTRY_191102.tsv', sep = '\t')
df8 = df8[['Client ID', 'Receiving Income (Entry)', 'Income Source (Entry)', 'Monthly Amount (Entry)']]
df_income_entry = pd.DataFrame(df8.groupby('Client ID', as_index=False).mean())
df9 = pd.read_csv('./data/INCOME_EXIT_191102.tsv', sep = '\t')
df9 = df9[['Client ID', 'ReceivingIncome (Exit)', 'Source of Income (Exit)', 'Monthly Amount (Exit)']]
df_income_exit = pd.DataFrame(df9.groupby('Client ID', as_index=False).mean())

In [205]:
df_income = pd.merge(df_income_exit, df_income_entry, on = ['Client ID'], how = 'left')
df_income = df_income.dropna(thresh = 2)
df_income.fillna(0, inplace = True)
df_income['changes'] = df_income['Monthly Amount (Exit)'] - df_income['Monthly Amount (Entry)']
df_clients = pd.merge(df_clients, df_income, on = ['Client ID'], how = 'left').fillna(0) #add income

In [206]:
df_udess = df5[['EE UID', 'Client ID', 'Prior Living Situation(43)', 
               'Total number of months homeless on the street, in ES or SH in the past three years(5168)', 
              'Housing Status(2703)', 'Domestic violence victim/survivor(341)', 'Does the client have a disabling condition?(1935)',
               'Covered by Health Insurance(4376)']]
df_udess = df_udess.rename(columns = {'Prior Living Situation(43)': 'prior_living_situation', 
               'Total number of months homeless on the street, in ES or SH in the past three years(5168)': 'months_on_street', 
              'Housing Status(2703)': 'housing_status', 'Domestic violence victim/survivor(341)': 'domestic_violence', 
                                      'Does the client have a disabling condition?(1935)': 'disabiltiy', 'Covered by Health Insurance(4376)': 'health_insurance'})
df_udess['months_on_street'] = df_udess['months_on_street'].replace(["Client doesn't know (HUD)",
                                                                     'Client refused (HUD)', "Data not collected (HUD)"],np.nan)
df_udess['prior_living_situation'] = df_udess['prior_living_situation'].replace('Data not collected (HUD)', np.nan).replace('Other (HUD)', np.nan)

In [210]:
df_case = pd.merge(df_case, df_udess, on = ['EE UID', 'Client ID'], how = 'left').merge(df_entry_exit, on = ['EE UID', 'Client ID'], how = 'left')
df_case = pd.merge(df_case, df_diab_exit, on = ['EE UID', 'Client ID'], how = 'left')
df_case = df_case.replace('More than 12 months (HUD)', '> 12').replace('One month (this time is the first month) (HUD)', '<1')

In [211]:
df_clients.to_csv('./data/CLIENTS.csv')
df_case.to_csv('./data/CASE.csv')
df_n.to_csv('./data/number.csv')

In [212]:
df5

,EE Provider ID,Entry Exit Provider Program Type Code,EE UID,Client Unique ID,Client ID,Client Location(4378),"Zip Code (of Last Permanent Address, if known)(1932)",Relationship to Head of Household(4374),Prior Living Situation(43),Length of Stay in Previous Place(1934),...,Did you stay less than 90 days?(5163),"On the night before did you stay on the streets, ES or SH?(5165)","Regardless of where they stayed last night - Number of times the client has been on the streets, in ES, or SH in the past three years including today(5167)","Total number of months homeless on the street, in ES or SH in the past three years(5168)",Housing Status(2703),Does the client have a disabling condition?(1935),Covered by Health Insurance(4376),Domestic violence victim/survivor(341),"If yes for Domestic violence victim/survivor, when experience occurred(1917)",Date of Birth(893)
0,Urban Ministries of Durham - Durham County - S...,Emergency Shelter (HUD),687901,pbkf09291954p610b236,397941,NC-502 Durham City and County CoC,27701,Self (head of household),"Staying or living in a friend's room, apartmen...",One year or longer (HUD),...,NaN,NaN,Two times (HUD),2,Category 1 - Homeless (HUD),Yes (HUD),Yes (HUD),No (HUD),NaN,9/29/1954
1,Urban Ministries of Durham - Durham County - S...,Emergency Shelter (HUD),687902,kdaf01071967k400d635,130335,NC-502 Durham City and County CoC,29033,Self (head of household),"Staying or living in a family member's room, a...","One month or more, but less than 90 days",...,NaN,NaN,Four or more times (HUD),More than 12 months (HUD),Category 1 - Homeless (HUD),Yes (HUD),Yes (HUD),Yes (HUD),More than a year ago (HUD),1/7/1967
2,Urban Ministries of Durham - Durham County - S...,Emergency Shelter (HUD),687903,smrf06211973s620m640,188933,NC-502 Durham City and County CoC,27703,Self (head of household),"Staying or living in a friend's room, apartmen...","One month or more, but less than 90 days",...,NaN,NaN,Four or more times (HUD),More than 12 months (HUD),Category 1 - Homeless (HUD),Yes (HUD),No (HUD),No (HUD),NaN,6/21/1973
3,Urban Ministries of Durham - Durham County - S...,Emergency Shelter (HUD),687904,abrm07251958a416b600,168290,NC-502 Durham City and County CoC,27603,Self (head of household),"Staying or living in a friend's room, apartmen...",One year or longer (HUD),...,NaN,NaN,Four or more times (HUD),Data not collected (HUD),Category 1 - Homeless (HUD),Yes (HUD),No (HUD),No (HUD),NaN,7/25/1958
4,Urban Ministries of Durham - Durham County - S...,Emergency Shelter (HUD),687905,wbom01251964w450b620,123122,NC-502 Durham City and County CoC,27510,Self (head of household),"Staying or living in a friend's room, apartmen...","One week or more, but less than one month",...,NaN,NaN,Four or more times (HUD),More than 12 months (HUD),Category 1 - Homeless (HUD),Yes (HUD),No (HUD),No (HUD),NaN,1/25/1964
5,Urban Ministries of Durham - Durham County - S...,Emergency Shelter (HUD),687907,tmjm08061985t520m230,363382,NC-502 Durham City and County CoC,23847,Self (head of household),"Staying or living in a family member's room, a...",One year or longer (HUD),...,NaN,NaN,Two times (HUD),6,Category 1 - Homeless (HUD),Yes (HUD),No (HUD),No (HUD),NaN,8/6/1985
6,Urban Ministries of Durham - Durham County - S...,Emergency Shelter (HUD),687909,ctnm05251964c642t534,165730,NC-502 Durham City and County CoC,27533,Self (head of household),"Emergency shelter, incl. hotel/motel paid for ...",One year or longer (HUD),...,NaN,NaN,Three times (HUD),Data not collected (HUD),Category 1 - Homeless (HUD),Yes (HUD),Yes (HUD),No (HUD),NaN,5/25/1964
7,Urban Ministries of Durham - Durham County - S...,Emergency Shelter (HUD),687911,rcam05241959r200c620,291392,NC-502 Durham City and County CoC,27701,Self (head of household),"Rental by client, with VASH housing subsidy (HUD)","90 days or more, but less than one year",...,NaN,NaN,Four or more times (HUD),More than 12 months (HUD),Category 1 - Homeless (HUD),No (HUD),Yes (HUD),No (HUD),NaN,5/24/1959
8,Urban Ministries 